# 初日COの集計

* 初日の作戦の参考用  
* 特にオチはない  

In [62]:
from __future__ import print_function, division 

import os 
import sys
import csv

import numpy as np
import pandas as pd
import sklearn.linear_model
import math

### AIWOLFPYのログ読み込み関数を使う

In [2]:
# add sys.path for aiwolfpy
sys.path.append("../AIWolfPy")

In [3]:
import aiwolfpy
import aiwolfpy.cash

In [41]:
def count_co(log_path):
    log_df = aiwolfpy.read_log(log_path)
    rolemap = {}
    for i in range(15):
        rolemap[i+1] = log_df["text"][i].split()[2]
    jinkei = {}
    jinkei_simple = {}
    for i in range(log_df.shape[0]):
        if log_df["day"][i] == 1 and log_df["type"][i] == "talk":
            if log_df["text"][i].split()[0] == "COMINGOUT":
                k = log_df["text"][i].split()[2] + "_" + rolemap[log_df["agent"][i]]
                k_simple = log_df["text"][i].split()[2]
                if k in jinkei.keys():
                    jinkei[k] += 1
                else:
                    jinkei[k] = 1
                if k_simple in jinkei_simple.keys():
                    jinkei_simple[k_simple] += 1
                else:
                    jinkei_simple[k_simple] = 1
        elif log_df["day"][i] > 1:
            break
    return str(sorted(jinkei_simple.items(), key=lambda x:x[0])), str(sorted(jinkei.items(), key=lambda x:x[0]))

とにかくCO発言をカウントする。重複は排除しない、いきなりDIVINEDというケースはカウントしない。

##### 勝ちチーム判定

In [69]:
def find_winner(log_path):
    df = pd.read_csv(log_path, header=None)
    return df.ix[df.shape[0]-1, 4]

In [70]:
find_winner(log_path)

'WEREWOLF'

適当なログでの実行サンプル

In [42]:
log_path = "../log/gat2017log15/000/000.log"
count_co(log_path)

("[('MEDIUM', 2), ('SEER', 2)]",
 "[('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1)]")

↑ 2-2, 正体は占いが真狂、霊媒が真狼

### 予備予選のログでやってみる

In [73]:
# collect logs
# note that this data has bias that cash(rev117) is in game
# cash(rev117) do not comingout when he is werewolf, bodyguard or villager.
# cash(rev117) comingout seer when he is seer or possessed, medium when medium


# 36*100 games
jinkei_dict = {}
for i in range(1, 37):
    for j in range(100):
        log_path = "../log/yobiyosen_2017/rev117/game-" + str(i) + "/" + "{0:03d}".format(j) + ".log"
        k, l = count_co(log_path)
        if k not in jinkei_dict.keys():
            jinkei_dict[k] = {} 
        if l not in jinkei_dict[k].keys():
            jinkei_dict[k][l] = {"N":0, "WEREWOLF":0, "VILLAGER":0}
        jinkei_dict[k][l]["N"] += 1
        jinkei_dict[k][l][find_winner(log_path)] += 1
    if i % 10 == 1 or i == 36:
        print("i = " + str(i) + " done")

i = 1 done
i = 11 done
i = 21 done
i = 31 done
i = 36 done


In [74]:
jinkei_dict

{"[('BODYGUARD', 1), ('MEDIUM', 1), ('SEER', 1)]": {"[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1)]": {'N': 4,
   'VILLAGER': 1,
   'WEREWOLF': 3},
  "[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1)]": {'N': 1,
   'VILLAGER': 0,
   'WEREWOLF': 1}},
 "[('BODYGUARD', 1), ('MEDIUM', 1), ('SEER', 2)]": {"[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1)]": {'N': 2,
   'VILLAGER': 1,
   'WEREWOLF': 1},
  "[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_MEDIUM', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)]": {'N': 2,
   'VILLAGER': 2,
   'WEREWOLF': 0},
  "[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 1)]": {'N': 1,
   'VILLAGER': 0,
   'WEREWOLF': 1},
  "[('BODYGUARD_BODYGUARD', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)]": {'N': 1,
   'VILLAGER': 1,
   'WEREWOLF': 0}},
 "[('BODYGUARD', 1), ('MEDIUM', 1), ('SEER', 3)]": {"[('BODYGUARD_BODYGUA

### 見づらいので、代表的なものを抽出

##### 占い1 霊媒0

In [77]:
k = "[('SEER', 1)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

186
15 [('SEER_WEREWOLF', 1)] 0.06666666666666667
99 [('SEER_POSSESSED', 1)] 0.1717171717171717
72 [('SEER_SEER', 1)] 0.4722222222222222


狂っぽい

##### 占い2 霊媒0

In [79]:
k = "[('SEER', 2)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

723
1 [('SEER_WEREWOLF', 2)] 1.0
120 [('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.475
108 [('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 1)] 0.2222222222222222
494 [('SEER_POSSESSED', 1), ('SEER_SEER', 1)] 0.31781376518218624


真狂が一番多い(494)けど、真狼(120)、狂狼(108)も意外といる

##### 占い3 霊媒0

In [80]:
k = "[('SEER', 3)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

635
6 [('SEER_SEER', 1), ('SEER_WEREWOLF', 2)] 0.5
627 [('SEER_POSSESSED', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.4098883572567783
2 [('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 2)] 0.0


さすがに真狂狼

##### 占い1 霊媒1

In [81]:
k = "[('MEDIUM', 1), ('SEER', 1)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

102
39 [('MEDIUM_MEDIUM', 1), ('SEER_SEER', 1)] 0.46153846153846156
4 [('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1)] 0.5
8 [('MEDIUM_MEDIUM', 1), ('SEER_WEREWOLF', 1)] 0.125
10 [('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1)] 0.0
2 [('MEDIUM_WEREWOLF', 1), ('SEER_WEREWOLF', 1)] 0.0
39 [('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1)] 0.15384615384615385


##### 占い2 霊媒1

In [82]:
k = "[('MEDIUM', 1), ('SEER', 2)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

448
39 [('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1)] 0.23076923076923078
14 [('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 1)] 0.14285714285714285
63 [('MEDIUM_MEDIUM', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.4126984126984127
12 [('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.5
73 [('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 1)] 0.2191780821917808
247 [('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1)] 0.39271255060728744


真狂ー真の狼3潜伏がこの中では多い。  
全体通して最もオーソドックスかと思った(仮に全部cashだとこうなる)が、そうでもない

##### 占い3 霊媒1

In [83]:
k = "[('MEDIUM', 1), ('SEER', 3)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

413
90 [('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.3888888888888889
1 [('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 2)] 0.0
4 [('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 2)] 0.25
1 [('MEDIUM_MEDIUM', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 2)] 1.0
1 [('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 2)] 0.0
316 [('MEDIUM_MEDIUM', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.40189873417721517


真狂狼ー真が一番多い。真狂狼ー狼も90あるけど、真霊媒はさすがにこうなったら潜伏しちゃダメだと思う。  
霊媒が真と思えたら、3分の1人狼の占いから吊る？  
黒が出ていたら霊媒鉄板護衛の黒出し吊り？    

##### 占い1 霊媒2

In [84]:
k = "[('MEDIUM', 2), ('SEER', 1)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

111
57 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1)] 0.19298245614035087
7 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_WEREWOLF', 1)] 0.42857142857142855
47 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1)] 0.5957446808510638


占いのバリエーションは色々あるけど、真占いはさすがにCOしよう。

##### 占い2 霊媒2

In [85]:
k = "[('MEDIUM', 2), ('SEER', 2)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

381
212 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1)] 0.4056603773584906
71 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 1)] 0.29577464788732394
98 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.5510204081632653


真狂ー真狼が一番多いけど、それ以外も結構ある。霊媒は真狼。やはりローラーか。

##### 占い3 霊媒2

In [86]:
k = "[('MEDIUM', 2), ('SEER', 3)]"
print(sum([jinkei_dict[k][l]["N"] for l in jinkei_dict[k].keys()]))
for l in jinkei_dict[k].keys():
    print(jinkei_dict[k][l]["N"], l, jinkei_dict[k][l]["VILLAGER"]/jinkei_dict[k][l]["N"])

336
5 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 2)] 0.6
328 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1), ('SEER_WEREWOLF', 1)] 0.5335365853658537
3 [('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_WEREWOLF', 2)] 0.3333333333333333


さすがに真狂狼ー真狼。霊媒ローラーから全ローラーか。村が勝ちやすいようだ。

##### カバレッジ

In [56]:
(186 + 723 + 635 + 102 + 448 + 413 + 111 + 381 + 336) / 3600

0.9263888888888889

##### 珍プレー

In [57]:
k = "[('MEDIUM', 2), ('SEER', 4), ('VILLAGER', 8)]"
print(sum([jinkei_dict[k][l] for l in jinkei_dict[k].keys()]), jinkei_dict[k])

16 {"[('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_SEER', 1), ('SEER_VILLAGER', 2), ('VILLAGER_VILLAGER', 8)]": 11, "[('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_POSSESSED', 1), ('SEER_VILLAGER', 2), ('SEER_WEREWOLF', 1), ('VILLAGER_VILLAGER', 8)]": 3, "[('MEDIUM_MEDIUM', 1), ('MEDIUM_WEREWOLF', 1), ('SEER_SEER', 1), ('SEER_VILLAGER', 2), ('SEER_WEREWOLF', 1), ('VILLAGER_VILLAGER', 8)]": 2}


村人が8CO?(このパターン以外もある)  
探してみる

In [60]:
# 36*100 games
jinkei_dict = {}
i = 1
for j in range(100):
    log_path = "../log/yobiyosen_2017/rev117/game-" + str(i) + "/" + "{0:03d}".format(j) + ".log"
    k, l = count_co(log_path)
    if "('VILLAGER', 8)" in k:
        print(i, j)

1 20
1 47
1 49


In [ ]:
0,status,1,WEREWOLF,ALIVE,kasuka
0,status,2,VILLAGER,ALIVE,sonoda
0,status,3,VILLAGER,ALIVE,tori
0,status,4,POSSESSED,ALIVE,AITKN
0,status,5,MEDIUM,ALIVE,wasabi
0,status,6,VILLAGER,ALIVE,carlo
0,status,7,VILLAGER,ALIVE,tsukammo
0,status,8,BODYGUARD,ALIVE,RyuZyu
0,status,9,VILLAGER,ALIVE,colun
0,status,10,VILLAGER,ALIVE,Udon
0,status,11,VILLAGER,ALIVE,megumish
0,status,12,SEER,ALIVE,Noppo
0,status,13,WEREWOLF,ALIVE,Litt1eGirl
0,status,14,VILLAGER,ALIVE,JuN1Ro
0,status,15,WEREWOLF,ALIVE,cash
0,whisper,0,0,13,COMINGOUT Agent[13] VILLAGER
0,whisper,1,0,1,COMINGOUT Agent[01] MEDIUM
0,whisper,2,0,15,Skip
0,whisper,3,1,1,Skip
0,whisper,4,1,13,Over
0,whisper,5,1,15,Skip
0,whisper,6,2,1,Skip
0,whisper,7,2,15,Over
0,whisper,8,2,13,Over
0,whisper,9,3,15,Over
0,whisper,10,3,1,Over
0,whisper,11,3,13,Over
0,divine,12,4,HUMAN
1,status,1,WEREWOLF,ALIVE,kasuka
1,status,2,VILLAGER,ALIVE,sonoda
1,status,3,VILLAGER,ALIVE,tori
1,status,4,POSSESSED,ALIVE,AITKN
1,status,5,MEDIUM,ALIVE,wasabi
1,status,6,VILLAGER,ALIVE,carlo
1,status,7,VILLAGER,ALIVE,tsukammo
1,status,8,BODYGUARD,ALIVE,RyuZyu
1,status,9,VILLAGER,ALIVE,colun
1,status,10,VILLAGER,ALIVE,Udon
1,status,11,VILLAGER,ALIVE,megumish
1,status,12,SEER,ALIVE,Noppo
1,status,13,WEREWOLF,ALIVE,Litt1eGirl
1,status,14,VILLAGER,ALIVE,JuN1Ro
1,status,15,WEREWOLF,ALIVE,cash
1,talk,0,0,1,VOTE Agent[02]
1,talk,1,0,5,COMINGOUT Agent[05] MEDIUM
1,talk,2,0,11,VOTE Agent[01]
1,talk,3,0,14,COMINGOUT Agent[14] SEER
1,talk,4,0,2,Skip
1,talk,5,0,12,COMINGOUT Agent[12] SEER
1,talk,6,0,10,VOTE Agent[07]
1,talk,7,0,8,VOTE Agent[09]
1,talk,8,0,7,VOTE Agent[14]
1,talk,9,0,4,COMINGOUT Agent[04] SEER
1,talk,10,0,13,Over
1,talk,11,0,9,VOTE Agent[03]
1,talk,12,0,3,VOTE Agent[10]
1,talk,13,0,6,VOTE Agent[01]
1,talk,14,0,15,VOTE Agent[02]
1,talk,15,1,9,Skip
1,talk,16,1,6,Skip
1,talk,17,1,12,DIVINED Agent[04] HUMAN
1,talk,18,1,5,Skip
1,talk,19,1,1,REQUEST(VOTE Agent[02])
1,talk,20,1,15,VOTE Agent[14]
1,talk,21,1,3,Skip
1,talk,22,1,4,DIVINED Agent[12] HUMAN
1,talk,23,1,14,COMINGOUT Agent[14] SEER
1,talk,24,1,7,ESTIMATE Agent[12] WEREWOLF
1,talk,25,1,13,Over
1,talk,26,1,8,Skip
1,talk,27,1,11,VOTE Agent[01]
1,talk,28,1,10,VOTE Agent[01]
1,talk,29,1,2,Skip
1,talk,30,2,1,VOTE Agent[05]
1,talk,31,2,5,Skip
1,talk,32,2,10,Over
1,talk,33,2,4,VOTE Agent[01]
1,talk,34,2,15,Skip
1,talk,35,2,8,ESTIMATE Agent[04] SEER
1,talk,36,2,9,ESTIMATE Agent[04] SEER
1,talk,37,2,13,Over
1,talk,38,2,11,VOTE Agent[01]
1,talk,39,2,6,Skip
1,talk,40,2,12,Over
1,talk,41,2,7,ESTIMATE Agent[04] WEREWOLF
1,talk,42,2,14,COMINGOUT Agent[14] VILLAGER
1,talk,43,2,2,Over
1,talk,44,2,3,ESTIMATE Agent[04] SEER
1,talk,45,3,11,VOTE Agent[01]
1,talk,46,3,4,REQUEST(VOTE Agent[01])
1,talk,47,3,12,Over
1,talk,48,3,1,REQUEST(VOTE Agent[05])
1,talk,49,3,5,Over
1,talk,50,3,8,Skip
1,talk,51,3,15,Skip
1,talk,52,3,9,Skip
1,talk,53,3,13,Over
1,talk,54,3,3,Skip
1,talk,55,3,14,COMINGOUT Agent[14] VILLAGER
1,talk,56,3,7,ESTIMATE Agent[11] WEREWOLF
1,talk,57,3,2,Over
1,talk,58,3,10,Over
1,talk,59,3,6,Over
1,talk,60,4,3,Skip
1,talk,61,4,15,Over
1,talk,62,4,10,Over
1,talk,63,4,12,Over
1,talk,64,4,5,Over
1,talk,65,4,4,ESTIMATE Agent[14] POSSESSED
1,talk,66,4,2,Over
1,talk,67,4,9,Skip
1,talk,68,4,1,COMINGOUT Agent[01] MEDIUM
1,talk,69,4,11,Over
1,talk,70,4,8,Skip
1,talk,71,4,14,COMINGOUT Agent[14] VILLAGER
1,talk,72,4,13,Over
1,talk,73,4,7,ESTIMATE Agent[13] WEREWOLF
1,talk,74,4,6,Over
1,talk,75,5,1,Skip
1,talk,76,5,11,Over
1,talk,77,5,14,COMINGOUT Agent[14] VILLAGER
1,talk,78,5,15,Over
1,talk,79,5,9,Over
1,talk,80,5,8,Over
1,talk,81,5,2,Over
1,talk,82,5,7,VOTE Agent[01]
1,talk,83,5,4,ESTIMATE Agent[12] POSSESSED
1,talk,84,5,5,Over
1,talk,85,5,3,Over
1,talk,86,5,12,Over
1,talk,87,5,10,Over
1,talk,88,5,6,VOTE Agent[02]
1,talk,89,5,13,Over
1,talk,90,6,2,Over
1,talk,91,6,8,Over
1,talk,92,6,9,Over
1,talk,93,6,10,Over
1,talk,94,6,1,Skip
1,talk,95,6,6,Skip
1,talk,96,6,11,Over
1,talk,97,6,13,Over
1,talk,98,6,14,COMINGOUT Agent[14] VILLAGER
1,talk,99,6,5,VOTE Agent[01]
1,talk,100,6,4,ESTIMATE Agent[14] POSSESSED
1,talk,101,6,12,Over
1,talk,102,6,3,Over
1,talk,103,6,15,Over
1,talk,104,6,7,ESTIMATE Agent[01] WEREWOLF
1,talk,105,7,8,Over
1,talk,106,7,9,Over
1,talk,107,7,13,Over
1,talk,108,7,7,ESTIMATE Agent[15] WEREWOLF
1,talk,109,7,10,Over
1,talk,110,7,4,VOTE Agent[05]
1,talk,111,7,14,COMINGOUT Agent[14] VILLAGER
1,talk,112,7,1,Over
1,talk,113,7,3,Over
1,talk,114,7,12,Over
1,talk,115,7,6,Skip
1,talk,116,7,5,ESTIMATE Agent[14] WEREWOLF
1,talk,117,7,2,Over
1,talk,118,7,15,Over
1,talk,119,7,11,VOTE Agent[01]
1,talk,120,8,2,Over
1,talk,121,8,9,Over
1,talk,122,8,14,COMINGOUT Agent[14] VILLAGER
1,talk,123,8,13,Over
1,talk,124,8,8,Over
1,talk,125,8,10,Over
1,talk,126,8,7,ESTIMATE Agent[12] SEER
1,talk,127,8,12,Over
1,talk,128,8,15,Over
1,talk,129,8,1,Over
1,talk,130,8,6,Over
1,talk,131,8,11,Over
1,talk,132,8,4,DIVINATION Agent[11]
1,talk,133,8,5,ESTIMATE Agent[13] VILLAGER
1,talk,134,8,3,Over
1,talk,135,9,7,Over
1,talk,136,9,3,Over
1,talk,137,9,2,Over
1,talk,138,9,6,Over
1,talk,139,9,13,Over
1,talk,140,9,5,ESTIMATE Agent[04] WEREWOLF
1,talk,141,9,15,Over
1,talk,142,9,12,Over
1,talk,143,9,10,Over
1,talk,144,9,8,Over
1,talk,145,9,1,Over
1,talk,146,9,11,Over
1,talk,147,9,14,COMINGOUT Agent[14] VILLAGER
1,talk,148,9,4,Skip
1,talk,149,9,9,Over
1,talk,150,10,8,Over
1,talk,151,10,5,Over
1,talk,152,10,13,Over
1,talk,153,10,7,Over
1,talk,154,10,6,Over
1,talk,155,10,10,Over
1,talk,156,10,4,Skip
1,talk,157,10,2,Over
1,talk,158,10,9,Over
1,talk,159,10,14,Over
1,talk,160,10,1,Over
1,talk,161,10,15,Over
1,talk,162,10,11,Over
1,talk,163,10,3,Over
1,talk,164,10,12,Over
1,talk,165,11,6,Over
1,talk,166,11,2,Over
1,talk,167,11,11,Over
1,talk,168,11,7,Over
1,talk,169,11,8,Over
1,talk,170,11,9,Over
1,talk,171,11,4,Over
1,talk,172,11,5,Over
1,talk,173,11,15,Over
1,talk,174,11,14,Over
1,talk,175,11,1,Over
1,talk,176,11,13,Over
1,talk,177,11,3,Over
1,talk,178,11,10,Over
1,talk,179,11,12,Over
1,vote,1,5
1,vote,2,12
1,vote,3,10
1,vote,4,1
1,vote,5,1
1,vote,6,2
1,vote,7,1
1,vote,8,9
1,vote,9,3
1,vote,10,1
1,vote,11,1
1,vote,12,15
1,vote,13,5
1,vote,14,1
1,vote,15,14
1,execute,1,WEREWOLF
1,divine,12,8,HUMAN
1,whisper,0,0,13,Over
1,whisper,1,0,15,Skip
1,whisper,2,1,13,Over
1,whisper,3,1,15,Skip
1,whisper,4,2,13,Over
1,whisper,5,2,15,Over
1,guard,8,4,POSSESSED
1,attackVote,13,14
1,attack,14,true

なるほど村人が占いCOして、撤回して延々村人CO。  
人狼が出にくくなってるのかな。  